In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import os
import numpy as np
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool
import concurrent.futures

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks\

/content/drive/MyDrive/Colab Notebooks


In [ ]:
"""# Getting the current work directory (cwd)
thisdir = os.getcwd()
snapshots = []

# r=root, d=directories, f = files
for r, d, f in os.walk(thisdir+"/tr2"):
    for file in f:
        if file.endswith(".snapshot"):
            snapshots.append(os.path.join(r, file))

def get_snap(snap):
    data = pd.DataFrame([])
    parser = feedparser.parse(snap)
    for post in parser.entries:
        try:
            title = post.title
            link = post.link
            date = post.published
            data = data.append(pd.DataFrame({'title': title, 'url': link,'date' : date}, index=[0]), ignore_index=True)
        except:
            continue
    return data
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for snap in snapshots:
        futures.append(executor.submit(get_snap, snap))
dfs = [df.result() for df in futures]
data = pd.concat(dfs)

data=data.drop_duplicates()

data.to_csv("turk_rss1.csv",index=False)
"""

In [ ]:
s = Session()
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
                         'AppleWebKit/537.36 (KHTML, like Gecko) '\
                         'Chrome/75.0.3770.80 Safari/537.36'}
# Add headers
s.headers.update(headers)

    
def get_user_need(page):
    try:
        article = s.get(page)
        soup = BeautifulSoup(article.content, 'lxml')
        x = [i for i in soup.find("script",text = re.compile(r'(?<="campaignName".").+?(?=\")'))][0]
        return re.search(r'(?<="campaignName".").+?(?=\")',x)[0]
    except:
        return np.nan
    
def get_text(url):
    try:
        article = s.get(url)
        soup = BeautifulSoup(article.content, 'lxml')
        body = soup.find(role='main')
        text = [p.text for p in body.find_all('p',dir="ltr")]
        return ''.join(text)
    except:
        return np.nan


In [ ]:
turk=pd.read_csv("turk_rss1.csv")

In [ ]:
turk.shape

(22243, 3)

In [ ]:
%%time
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    futures2 = []
    for page in turk["url"]:
        futures.append(executor.submit(get_user_need, page))
        futures2.append(executor.submit(get_text, page))

In [ ]:
userneed = [df.result() for df in futures]
turk["User Need"] = userneed
turk["User Need"] = turk["User Need"].apply(lambda s: re.sub(r"(WS - )", "", s) if pd.notnull(s) else s)

In [ ]:
text = [df.result() for df in futures2]
turk["text"] = text

In [ ]:
turk = turk.dropna(subset=["User Need","text"])

In [50]:
#turk.tail(50)

In [47]:
def categorize(x):
    if 'spor' in x:
        return "sports"

In [48]:
turk['category'] = turk["url"].apply (lambda x: categorize(x))

In [49]:
turk.head()

,title,url,date,User Need,text,category
0,Irak'ın Ramadi kentinde 75 Türk 'mahsur',http://www.bbc.co.uk/turkce/haberler/2014/01/1...,2014-01-01T22:09:47+02:00,Update me,"Irak'ın Ramadi kentinde Irak ordusu, Arap aşir...",None
1,Kuzey Kore liderinden idam mesajı: Hizipçi pis...,http://www.bbc.co.uk/turkce/haberler/2014/01/1...,2014-01-01T20:42:20+02:00,Update me,"Kuzey Kore lideri Kim Jong-un, yılbaşı mesajın...",None
2,AB dönem başkanlığı Yunanistan'da,http://www.bbc.co.uk/turkce/haberler/2014/01/1...,2014-01-01T16:24:04+02:00,Update me,Yunanistan bugün AB dönem başkanlığını devral...,None
3,Lübnan'da radikal İslamcı lider tutuklandı,http://www.bbc.co.uk/turkce/haberler/2014/01/1...,2014-01-01T13:28:57+02:00,Update me,Lübnan'da geçen Kasım ayında İran Büyükelçiliğ...,None
4,Suudi şarkıcıya zina iftirasına 80 kırbaç,http://www.bbc.co.uk/turkce/haberler/2014/01/1...,2014-01-01T15:24:40+02:00,Update me,Suudi Arabistan'da Arap dünyasının ünlü kadın ...,None


In [51]:
turk.to_csv("turkcheckpoint.csv",index=False)